In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math

In [14]:
class InputEmbeddings(nn.Module):
  def __init__(self, vocab_size, d_model=512):
    super().__init__()
    self.vocab_size = vocab_size
    self.d_model = d_model
    self.embedding = nn.Embedding(vocab_size, d_model)

  def forward(self, x):
    return self.embedding(x) * math.sqrt(self.d_model)

In [15]:
class PositionalEncoding(nn.Module):
  def __init__(self, seq_size, d_model=512):
    self.seq_size = seq_size
    self.d_model = d_model

    self.pos_enc = torch.zeros(seq_size, d_model)
    pos = torch.arange(0, seq_size).view(-1, 1).float()
    common = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

    self.pos_enc[:, 0::2] = torch.sin(pos * common)
    self.pos_enc[:, 1::2] = torch.cos(pos * common)
    self.pos_enc.view(1, seq_size, -1).requires_grad_(False)

  def forward(self, x):
    return x + self.pos_enc[:, :x.size(1)] # x.size(1) is the sequence length, pos_enc is always fixed


In [16]:
class FeedForward(nn.Module):
  def __init__(self, d_model=512, d_ff=2048, drop=0.1):
    super().__init__()
    self.d_model = d_model
    self.d_ff = d_ff
    
    self.net = nn.Sequential(
      nn.Linear(self.d_model, self.d_ff),
      nn.Relu(),
      nn.Dropout(drop),
      nn.Linear(self.d_ff, self.d_model),
    )
  
  def forward(self, x):
    return self.net(x)

In [17]:
class MultiHeadAttentionBlock(nn.Module):

    def __init__(self, d_model: int, h: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model # Embedding vector size
        self.h = h # Number of heads
        # Make sure d_model is divisible by h
        assert d_model % h == 0, "d_model is not divisible by h"

        self.d_k = d_model // h # Dimension of vector seen by each head
        self.w_q = nn.Linear(d_model, d_model, bias=False) # Wq
        self.w_k = nn.Linear(d_model, d_model, bias=False) # Wk
        self.w_v = nn.Linear(d_model, d_model, bias=False) # Wv
        self.w_o = nn.Linear(d_model, d_model, bias=False) # Wo
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        d_k = query.shape[-1]
        # Just apply the formula from the paper
        # (batch, h, seq_len, d_k) --> (batch, h, seq_len, seq_len)
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            # Write a very low value (indicating -inf) to the positions where mask == 0
            attention_scores.masked_fill_(mask == 0, -1e9)
        attention_scores = attention_scores.softmax(dim=-1) # (batch, h, seq_len, seq_len) # Apply softmax
        if dropout is not None:
            attention_scores = dropout(attention_scores)
        # (batch, h, seq_len, seq_len) --> (batch, h, seq_len, d_k)
        # return attention scores which can be used for visualization
        return (attention_scores @ value), attention_scores

    def forward(self, q, k, v, mask):
        query = self.w_q(q) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        key = self.w_k(k) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        value = self.w_v(v) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)

        # (batch, seq_len, d_model) --> (batch, seq_len, h, d_k) --> (batch, h, seq_len, d_k)
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

        # Calculate attention
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)
        
        # Combine all the heads together
        # (batch, h, seq_len, d_k) --> (batch, seq_len, h, d_k) --> (batch, seq_len, d_model)
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)

        # Multiply by Wo
        # (batch, seq_len, d_model) --> (batch, seq_len, d_model)  
        return self.w_o(x)